# Dataset: CIFAR-10

## Code Used: https://github.com/drgripa1/resnet-cifar10


# Setting up environment and importing libraries

## Requirements:


1.   Keras
2.   Code from: https://github.com/drgripa1/resnet-cifar10
3.   Numpy
4.   Matplotlib
5.   OpenCV



In [6]:
# https://keras.io/
!git clone https://github.com/drgripa1/resnet-cifar10.git

fatal: destination path 'resnet-cifar10' already exists and is not an empty directory.


In [0]:
from keras.datasets import cifar10
from classification_models.resnet import ResNet18, preprocess_input
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from keras import optimizers
from keras.callbacks import ModelCheckpoint

In [0]:
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing import image
import keras.backend as K
import cv2
import sys

# Using the code's parameters, train models
## First get dataset using the repo's code

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
print(np.mean(x_test[0]))
print(np.mean(x_train[0]))

108.38606770833333
103.447265625


In [0]:
# normalize inputs from 0-255 to 0.0-1.0
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0

In [0]:
print(np.mean(x_test[0]))
print(np.mean(x_train[0]))

0.42504343
0.40567556


### Generate models using repo's code

In [ ]:
batch_size = 128
n_classes = 10 #Because CIFAR10 has 10 classes
epochs = 20
!mkdir checkpoint_dir
!python train.py --n 3 --checkpoint_dir checkpoint_dir
!python test.py --n 3 --checkpoint_dir checkpoint_dir/model_final.pth

# Architecture Design: ResNet
## different optimization strategies: SGD, momentum SGD with momentum term equal to 0.9, and ADAM
### First of all, SGD
Note that, we will conclude the train and test error after each method

In [0]:
base_model = ResNet18(input_shape=(32,32,3), weights='imagenet', include_top=False)
x = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation='softmax')(x)
model = keras.models.Model(inputs=[base_model.input], outputs=[output])
sgd = optimizers.SGD(lr=0.1, decay=0.0001, nesterov=True)
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 32, 32, 3)    9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_19 (ZeroPadding2 (None, 38, 38, 3)    0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 16, 16, 64)   9408        zero_padding2d_19[0][0]          
__________________________________________________________________________________________________
bn0 (Batch

In [0]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 21s 2ms/step
Test loss: 2.4171396324157715
Test accuracy: 0.1


### Next, SGD with momentum = 0.9

In [ ]:
sgd = optimizers.SGD(lr=0.1, decay=0.0001, momentum=0.9, nesterov=True)
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Test loss: 1.6304093204734795
Test accuracy:0.108


### Finally, ADAM

In [ ]:
adam = optimizers.ADAM(lr=0.1, decay=0.0001, nesterov=True)
model.compile(optimizer='ADAM', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Test loss: 1.6304093204734795
Test accuracy:0.121


### Details of  train and test loss versus iterations, Play around with learning rates

In [0]:
import math
# learning rate schedule
def step_decay(epoch):
	initial_lrate = 0.1
	drop = 0.7
	epochs_drop = 7.0
	lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lrate

In [0]:
for i in range(1,26):
  print("Epoch "+str(i)+ " : ",step_decay(i))

Epoch 1 :  0.1
Epoch 2 :  0.1
Epoch 3 :  0.1
Epoch 4 :  0.1
Epoch 5 :  0.1
Epoch 6 :  0.06999999999999999
Epoch 7 :  0.06999999999999999
Epoch 8 :  0.06999999999999999
Epoch 9 :  0.06999999999999999
Epoch 10 :  0.06999999999999999
Epoch 11 :  0.06999999999999999
Epoch 12 :  0.06999999999999999
Epoch 13 :  0.048999999999999995
Epoch 14 :  0.048999999999999995
Epoch 15 :  0.048999999999999995
Epoch 16 :  0.048999999999999995
Epoch 17 :  0.048999999999999995
Epoch 18 :  0.048999999999999995
Epoch 19 :  0.048999999999999995
Epoch 20 :  0.03429999999999999
Epoch 21 :  0.03429999999999999
Epoch 22 :  0.03429999999999999
Epoch 23 :  0.03429999999999999
Epoch 24 :  0.03429999999999999
Epoch 25 :  0.03429999999999999


In [0]:
from keras.callbacks import LearningRateScheduler
lrate = LearningRateScheduler(step_decay)

In [0]:
epochs = 25

In [0]:
model.fit(x_train, y_train, batch_size=batch_size, validation_data=(x_test, y_test), epochs=epochs, verbose=1,callbacks=[lrate,checkpoint])

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 48s 950us/step - loss: 0.8804 - acc: 0.7004 - val_loss: 0.7383 - val_acc: 0.7479

Epoch 00001: val_acc improved from -inf to 0.74790, saving model to Assignment_5.hdf5
Epoch 2/25
50000/50000 [==============================] - 16s 321us/step - loss: 0.5095 - acc: 0.8239 - val_loss: 0.6738 - val_acc: 0.7710

Epoch 00002: val_acc improved from 0.74790 to 0.77100, saving model to Assignment_5.hdf5
Epoch 3/25
50000/50000 [==============================] - 16s 322us/step - loss: 0.3639 - acc: 0.8726 - val_loss: 0.6230 - val_acc: 0.8002

Epoch 00003: val_acc improved from 0.77100 to 0.80020, saving model to Assignment_5.hdf5
Epoch 4/25
50000/50000 [==============================] - 16s 325us/step - loss: 0.2617 - acc: 0.9084 - val_loss: 0.8335 - val_acc: 0.7603

Epoch 00004: val_acc did not improve from 0.80020
Epoch 5/25
50000/50000 [==============================] - 16s 320us/step - l

In [0]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 192us/step
Test loss: 1.0762726390004158
Test accuracy: 0.8371


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("ResNet_CIFAR10_Weights_Best.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
# Save the trained model in to .h5 format
model.save("ResNet_CIFAR10_Model_Best.h5")
print("Saved model to disk")

Saved model to disk


As a conclusion, we found out that ADAM was the best followed by Momentum SGD and finally Vanilla SGD. The leraning rate was optimized at 0.1, as for Momentum SGD tested.